"""Modeling Alternative Assets in LUSD

* Demonstrating how LUSID can be used to manage a Private Equity fund.

Attributes
----------
holdings
transactions
transaction configuration
"""

# Modelling Alternative Assets in LUSID

* In this notebook we'll look at how LUSID can be used to manage a Private Equity fund.

* Specifically, we'll look at processing commitments, making capital calls and distributing profits to investors. 

* We'll also look at running valuations.

* Throughout the demo, we'll consider the life of a PE fund over a 5 day period
    * On Day 1, we'll look at the posting of commitments, this is money that an investor has agreed to contribute into the investment fund
    * On Day 2, we'll make a call against that committed capital
    * On Day 3, we make a real estate asset
    * On Day 4, we get some income from that asset
    * On Day 5, we distribute this income to investors

## LUSID Web UI

In [ ]:
import os

api_url = os.getenv("FBN_LUSID_API_URL")

domain_prefix = api_url[:-3]

home = domain_prefix + "app/dashboard/holdings?scope=AltBook&code=European-Infrastructure-Fund&entityType=Portfolio&effectiveDate=2021-03-01"

print(home)

This notebook sets up an example Portfolio which can be viewed in the LUSID Web UI with the link above.

### Day 1 - A Transaction representing a Investor's Commitment to the fund 

<b>Open the Holdings tab of LUSID using the link above</b>

* There are two equal and opposite holdings visible:
  * The first line represents the 10m EUR cash which has been committed to the fund by Investor 1
  * The second line represents the 10m EUR liability we have to Investor 1
  
<b>Open the Transactions tab of LUSID</b>

* Moving over to the <b>Transaction</b>, we can see the Transaction which created these two holdings along with the <b>Transaction Type</b> associated with the transaction.

* As noted earlier, the <b>Transaction Type</b> describes the economic movements which determine the holding in the <b>Holding</b> statement.

* What do I mean by that? Here we have a Transaction Type of "Commitment" which tells LUSID to generate two cash movements:
    * A commitment balance asset of 10m EUR to the portfolio
    * A commitment balance liability of 10m EUR to Investor 1
<br>

### Day 2 - A Capital Call Transaction

* Moving onto Day 2, we see two new "Capital" lines which represent the capital calls against the Commitment

<b>Open the Holdings tab of LUSID for 2 March 2021</b>

* The capital calls can be idenified as those with SubscriptionType=Capital

* Again we can see:
  * A capital call asset of 5m EUR
  * A capital call liability of 5m EUR to the investor
<br>

<b>Open the Transactions tab of LUSID</b>

* Moving back into the <b>Transaction</b> tab, we can see we have a capital call transaction

* And this "CapitalCall" transaction uses a different <b>Transaction Type</b>
 
* The transaction type here will move cash from the Committed bucket into the Capital bucket

### Day 3 - An investment Transaction

* On Day 3, lets use some of that capital to make an investment

<b>Open the Holdings tab of LUSID for 3 March 2021</b>

* In this example, we have purchased a shopping center in The Hague

* You can see the asset reflected in our holdings statement

<b>Click into the LUID to pull up instrument details</b>

* Click into the LUID and then the definition and you can see details about the asset:
  * We can see it is a shopping centre
  * It has a unqiue client ID
  * It has a rating of B1
  * We can see it has an average retail lease of 2.3 years
<br>

* We should note here that these fields are entirely customisable - you can populate them with any documents you please.

* Lets look at the transactions again

<b>Open the Transactions tab of LUSID</b>

* Here we see a new transaction for the initial investment

* Again the transation type "InitialInvestment" has two movements: 
  * It decreases the capital amount cash balance
  * It records the purchase of the shopping centre
<br>

### Day 4 - An income Transaction

* On Day 4 the shopping centre produces 50,000 EUR of income

<b>Open the Holdings tab of LUSID for 4 March 2021</b>

* Looking at the holding statement we see 50,000 EUR of income

<b>Open the Transactions tab of LUSID</b>

* Clicking through to the transactions we see that the income has been recorded against the shopping centre LUID

* And the "Distribution" <b>Transaction Type</b> tells us that this transaction should increase the cash in the income bucket


### Day 5 - Pay income to investors

* Finally on Day 5 we pay out income to investors

<b>Open the Holdings tab of LUSID for 5 March 2021</b>

* On the holding report you can see a liability of cash recorded against Investor 1

<b>Open the Transactions tab of LUSID</b>

* Clicking into the transaction you can see we have a distribution


### Valuations

* We can also run valuations on alternative assets in LUSID

* Using LUSID's flexible recipe system we can load prices from any external source and value our alternative assets alongside traditional liquid assets

* In the following example we have valued the shopping centre asset at 5m EUR using an external price feed

In [ ]:
import os

api_url = os.getenv("FBN_LUSID_API_URL")

domain_prefix = api_url[:-3]

home = domain_prefix + "app/dashboard/valuations?scope=AltBook&code=European-Infrastructure-Fund&entityType=Portfolio&recipeScope=AltBook&recipeCode=alternatives&groupBy=queryable.Value(Instrument%2Fdefault%2FName)&effectiveDate=2021-03-05"

print(home)

## Setup

In [ ]:
# Import LUSID
import lusid.models as models
from lusidjam import RefreshingToken
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import pandas as pd
import json
import lusid
import uuid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")


print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.6.6606.0


In [ ]:
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

In [ ]:
scope = "AltBook" 

## Create properties to hold transaction information

In [ ]:
property_codes = ["SubscriptionType", "InvestorId", "FundCode", "OperationStage"]
property_keys = {}

for property_code in property_codes:
    
    try:

        # Create our request to define a new property
        request = models.CreatePropertyDefinitionRequest(
            domain='Transaction',
            scope=scope,
            code=property_code,
            value_required=False,
            display_name=property_code,
            data_type_id=models.ResourceId(scope='system', code='string'))

        # Call LUSID to create our new property
        response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
            create_property_definition_request=request)

        # Grab the key off the response to use when referencing this property in other LUSID calls
        property_keys[property_code] = response.key
        
    except:
        pass

## Create an investor portfolio

In [ ]:
# Set the code of your portfolio
investor_portfolio_code = 'Investor1'

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

try:

    # Build your request to create your portfolio
    request = models.CreateTransactionPortfolioRequest(
        display_name=investor_portfolio_code,
        code=investor_portfolio_code,
        base_currency='EUR',
        description=None,
        created=portfolio_creation_date,
        corporate_action_source_id=None,
        accounting_method='AverageCost',
        sub_holding_keys=[f"Transaction/{scope}/SubscriptionType", f"Transaction/{scope}/FundCode"],
        properties=None)

    # Call LUSID to create your portfolio
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=request)
    
except lusid.ApiException as e:
    print(e)


(400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 21 Mar 2021 19:59:24 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4968', 'X-Rate-Limit-Reset': '2021-03-21T20:00:05.4482464Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HM7BI6MCCN63:00000006', 'lusid-meta-correlationId': '0HM7BI6MCCN63:00000006', 'lusid-meta-duration': '188', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-uri https://lusid.report-uri.com/r/d/csp/enforce", 'X-Fram

## Create a fund portfolio

In [ ]:
# Set the code of your portfolio
fund_portfolio_code = 'European-Infrastructure-Fund'

# Set the creation date of your portfolio 
portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=1052)

try:

    # Build your request to create your portfolio
    request = models.CreateTransactionPortfolioRequest(
        display_name=fund_portfolio_code,
        code=fund_portfolio_code,
        base_currency='EUR',
        description=None,
        created=portfolio_creation_date,
        corporate_action_source_id=None,
        accounting_method='AverageCost',
        sub_holding_keys=[f"Transaction/{scope}/SubscriptionType", f"Transaction/{scope}/InvestorId"],
        properties=None)

    # Call LUSID to create your portfolio
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=request)
    
except lusid.ApiException as e:
    print(e)


(400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 21 Mar 2021 19:59:25 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4967', 'X-Rate-Limit-Reset': '2021-03-21T20:00:05.4482464Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HM7BI6RF5RB5:00000008', 'lusid-meta-correlationId': '0HM7BI6RF5RB5:00000008', 'lusid-meta-duration': '126', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-uri https://lusid.report-uri.com/r/d/csp/enforce", 'X-Fram

## Create new transaction types for alts - Commitment

You define two transaction types below.

- One for the fund which indicates that you have a commitment from the investor associated with the transaction
- One for the investor which indicates that this investor has a commitment to the fund associated with the transaction

In [ ]:
new_transaction_config = [models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type='Commitment',
                    description='The commitment of funds by an investor',
                    transaction_class='FundOperations',
                    transaction_group='FundOperations-Investor',
                    transaction_roles='Longer')
            ],
            movements=[
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashCommitment',
                    side='Side2',
                    direction=1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Committed")
                    ]
                )
            ],
            properties=None),models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type='Commitment',
                    description='The commitment of funds by an investor',
                    transaction_class='FundOperations',
                    transaction_group='FundOperations-Fund',
                    transaction_roles='Longer')
            ],
            movements=[
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashCommitment',
                    side='Side2',
                    direction=-1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Committed")
                    ]),
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashCommitment',
                    side='Side2',
                    direction=1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Committed"),
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/InvestorId",
                            set_to="")
                    ]
                )
            ],
            properties=None)]

new_txn_config = upsert_transaction_type_alias(api_factory, new_transaction_config=new_transaction_config)

## Create new transaction types for alts - Capital Call

- One for the fund which moves the committed balance for the investor associated with the capital call to a capital balance which can be used for transactions
- One for the investor which moves the committed balance for the fund associated with the capital call to a capital balance which has been provided to the fund to use for transactions

In [ ]:
new_transaction_config = [models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type='CapitalCall',
                    description='The provision of funds by an investor in response to a capital call',
                    transaction_class='FundOperations',
                    transaction_group='FundOperations-Investor',
                    transaction_roles='LongShorter')
            ],
            movements=[
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashSettlement',
                    side='Side2',
                    direction=-1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Committed")
                    ]),
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashCommitment',
                    side='Side2',
                    direction=1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Capital")
                    ]
                )
            ],
            properties=None),models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type='CapitalCall',
                    description='The provision of funds by an investor in response to a capital call',
                    transaction_class='FundOperations',
                    transaction_group='FundOperations-Fund',
                    transaction_roles='Longer')
            ],
            movements=[
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashSettlement',
                    side='Side2',
                    direction=1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Committed")
                    ]),
                 models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashSettlement',
                    side='Side2',
                    direction=-1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Capital")
                    ]),
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashSettlement',
                    side='Side2',
                    direction=-1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Committed"),
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/InvestorId",
                            set_to="")
                    ]),
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashSettlement',
                    side='Side2',
                    direction=1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Capital"),
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/InvestorId",
                            set_to="")
                    ]
                )
            ],
            properties=None)]

new_txn_config = upsert_transaction_type_alias(api_factory, new_transaction_config=new_transaction_config)

## Create new transaction types for alts - Distribution

You define two transaction types below.

- One for the fund which generates a distribution of income against the investor associated with the distribution
- One for the investor which receives a distribution of income from to fund associated with the distribution

In [ ]:
# Call LUSID to create your transaction type

new_transaction_config = [models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type='Distribution',
                    description='Income distributed from a fund',
                    transaction_class='FundOperations',
                    transaction_group='FundOperations-Investor',
                    transaction_roles='Longer')
            ],
            movements=[
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashCommitment',
                    side='Side2',
                    direction=1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Income")
                    ]
                )
            ],
            properties=None), models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type='Distribution',
                    description='Income distributed from a fund',
                    transaction_class='FundOperations',
                    transaction_group='FundOperations-Fund',
                    transaction_roles='Shorter')
            ],
            movements=[
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashCommitment',
                    side='Side2',
                    direction=-1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Income")
                    ]
                )
            ],
            properties=None)]

new_txn_config = upsert_transaction_type_alias(api_factory, new_transaction_config=new_transaction_config)

## Create new transaction types for alts - Initial Investment


An initial investment in an asset.

This is only a single transaction type which reduces the amount of capital cash and increases the amount of the asset held in the fund associated with the transaction.

In [ ]:
new_transaction_config=  [models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type='InitialInvestment',
                    description='The purchase of a security',
                    transaction_class='AssetTransactions',
                    transaction_group='AssetTransactions',
                    transaction_roles='Longer')
            ],
            movements=[
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='StockMovement',
                    side='Side1',
                    direction=1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to=""),
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/InvestorId",
                            set_to="Capital")
                    ]
                ),
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashCommitment',
                    side='Side2',
                    direction=-1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Capital"),
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/InvestorId",
                            set_to="")
                    ]
                )
            ],
            properties=None)]
    
new_txn_config = upsert_transaction_type_alias(api_factory, new_transaction_config=new_transaction_config)

## Create new transaction type for alts - Distribution

In [ ]:
new_transaction_config = [models.TransactionConfigurationDataRequest(
            aliases=[
                models.TransactionConfigurationTypeAlias(
                    type='Distribution',
                    description='Income distributed from an infrastructure or real asset',
                    transaction_class='AssetTransactions',
                    transaction_group='AssetTransactions',
                    transaction_roles='Longer')
            ],
            movements=[
                models.TransactionConfigurationMovementDataRequest(
                    movement_types='CashCommitment',
                    side='Side2',
                    direction=1,
                    properties=None,
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/SubscriptionType",
                            set_to="Income"),
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/InvestorId",
                            set_to="")
                    ]
                )
            ],
            properties=None)]

new_txn_config = upsert_transaction_type_alias(api_factory, new_transaction_config=new_transaction_config)

## Define dates to work with

In [ ]:
current_time = datetime(year=2021, month=3, day=6, tzinfo=pytz.UTC)
day_1 = current_time - timedelta(days=5)
day_2 = current_time - timedelta(days=4)
day_3 = current_time - timedelta(days=3)
day_4 = current_time - timedelta(days=2)
day_5 = current_time - timedelta(days=1)

## Create a function to post transaction

In [ ]:
def subscription_transaction(date, units, transaction_type, investor_portfolio_code, fund_portfolio_code, unique_transaction_id, status=None):
    """
    This function creates subscription transactions for committing to a fund as well as provisioning capital via
    a capital call. 
    
    param date (datetime): The datetime to upsert the transactions at
    param units (float): The number of units in the transaction
    param transaction_type (str): The code for the transaction type for this subscription transaction
    param investor_portfolio_code (str): The code for the investor's portfolio
    param fund_portfolio_code (str): The code for the fund portfolio
    
    return unique_transaction_id (str): The unique id for the transactions
    """
    
    transactions = {}
    
    unique_transaction_id = unique_transaction_id
    
    investor_properties = {
        f"Transaction/{scope}/FundCode": models.ModelProperty(
            key=f"Transaction/{scope}/FundCode",
            value=models.PropertyValue(
                label_value=fund_portfolio_code)
        )
    }
    
    if type(status) is not type(None):
        investor_properties[f"Transaction/{scope}/OperationStage"] = models.ModelProperty(
            key=f"Transaction/{scope}/OperationStage",
            value=models.PropertyValue(
                label_value=status)
        )
            
            
    # Construct the transaction for the investor's portfolio
    transactions[investor_portfolio_code] = models.TransactionRequest(
        transaction_id=unique_transaction_id,
        type=transaction_type,
        instrument_identifiers={
            'Instrument/default/Currency': 'EUR'
        },
        transaction_date=date,
        settlement_date=date,
        units=units,
        transaction_price=models.TransactionPrice(
            price=1,
            type='Price'),
        total_consideration=models.CurrencyAndAmount(
            amount=units,
            currency='EUR'),
        source='FundOperations-Investor',
        transaction_currency='EUR',
        properties=investor_properties)
    
    # Construct the transaction for the fund's portfolio
    transactions[fund_portfolio_code] = models.TransactionRequest(
        transaction_id=unique_transaction_id,
        type=transaction_type,
        instrument_identifiers={
            'Instrument/default/Currency': 'EUR'
        },
        transaction_date=date,
        settlement_date=date,
        units=units,
        transaction_price=models.TransactionPrice(
            price=1,
            type='Price'),
        total_consideration=models.CurrencyAndAmount(
            amount=units,
            currency='EUR'),
        source='FundOperations-Fund',
        transaction_currency='EUR',
        properties={
            f"Transaction/{scope}/InvestorId" : models.ModelProperty(
                key=f"Transaction/{scope}/InvestorId",
                value=models.PropertyValue(
                    label_value=investor_portfolio_code)
            )
        })

    # Loop over the portfolios
    for portfolio_code, transaction in transactions.items():
        
        # Call LUSID to upsert the transactions
        response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transaction_request=[transaction])

        # Print the response from LUSID using pretty formatting 
#         prettyprint.transactions_response(
#             response,
#             scope,
#             portfolio_code)
    
    return 

## Day 1 - Investor commits EUR 10,000,000 to the fund

In [ ]:
subscription_transaction(
    date=day_1, 
    units=10000000, 
    transaction_type='Commitment',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    unique_transaction_id="funds_in_1")

## Day 2 - The fund makes a capital call of EUR 5,000,000 to the investor

In [ ]:
subscription_transaction(
    date=day_2, 
    units=5000000, 
    transaction_type='CapitalCall',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    unique_transaction_id="capital_call_1")

## Day 3 - The fund purchases an alternative physical asset

In [ ]:
# Create the bespoke hedge contract definition
shopping_centre_asset = {
    "identifier": "SYD_241232",
    "no_units": 76,
    "average_retail_lease": "2.3 years",
    "term": "5yr",
    "freq": "Quarterly",
    "rating": "B1"
}

# Create the definition for your instrument, attaching the bespoke contract
shopping_centre_asset_instrument = models.InstrumentDefinition(
    name='The_Hague_Shopping_Centre',
    identifiers={
        'ClientInternal': models.InstrumentIdValue(
            value=shopping_centre_asset["identifier"])},
    definition=models.ExoticInstrument(
        instrument_format=models.InstrumentDefinitionFormat(
        source_system='ClientSystemA',
        vendor='ClientA',
        version='0.0.1'
        ),
        content=json.dumps(shopping_centre_asset),
        instrument_type='ExoticInstrument'            
    )    
)

response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body={"shopping_centre": shopping_centre_asset_instrument})


In [ ]:
transaction = models.TransactionRequest(
    transaction_id='Purchase_22319041341',
    type='InitialInvestment',
    instrument_identifiers={
        'Instrument/default/ClientInternal': 'SYD_241232'
    },
    transaction_date=day_3,
    settlement_date=day_3,
    units=1,
    transaction_price=models.TransactionPrice(
        price=5000000,
        type='Price'),
    total_consideration=models.CurrencyAndAmount(
        amount=5000000,
        currency='EUR'),
    source='AssetTransactions',
    transaction_currency='EUR')

# Call LUSID to upsert your transactions for this portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope,
    code=fund_portfolio_code,
    transaction_request=[transaction])

## Day 4 - Income is generated from the asset

In [ ]:
transaction = models.TransactionRequest(
    transaction_id='Income_22319041341',
    type='Distribution',
    instrument_identifiers={
        'Instrument/default/ClientInternal': 'SYD_241232'
    },
    transaction_date=day_4,
    settlement_date=day_4,
    units=1,
    transaction_price=models.TransactionPrice(
        price=50000,
        type='Price'),
    total_consideration=models.CurrencyAndAmount(
        amount=50000,
        currency='EUR'),
    source='AssetTransactions',
    transaction_currency='EUR',
    properties={f"Transaction/{scope}/OperationStage": models.ModelProperty(
        key=f"Transaction/{scope}/OperationStage",
        value=models.PropertyValue(
            label_value='Approved')
    )})

# Call LUSID to upsert your transactions for this portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope,
    code=fund_portfolio_code,
    transaction_request=[transaction])

## Day 5 - Income is distributed to investors

In [ ]:
subscription_transaction(
    date=day_5, 
    units=25000, 
    transaction_type='Distribution',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    unique_transaction_id="distribution_1",
    status='Approved')

## Create a recipe and run a valuation

In [ ]:
start_date = datetime(year=2021, month=1, day=1)
num_of_days = 90
days = [(start_date + timedelta(days=x)).strftime(format="%Y-%m-%d") for x in range(num_of_days)]

In [ ]:
quotes_for_upsert = {}

prices = [5000000] * len(days)

daily_prices = tuple(zip(days, prices))

for date, price in daily_prices:

    quotes_for_upsert["quotes_request_" + "SYD_241232" + "_" + date.replace("-","")] = models.UpsertQuoteRequest(
                quote_id=models.QuoteId(
                    quote_series_id=models.QuoteSeriesId(
                        provider='Lusid',
                        instrument_id="SYD_241232",
                        instrument_id_type="ClientInternal",
                        quote_type="Price",
                        field="mid"
                    ),
                    effective_at=date),
                metric_value=models.MetricValue(
                    value=price,
                    unit="EUR"
            ))

In [ ]:
response = quotes_api.upsert_quotes(scope=scope, request_body=quotes_for_upsert)

In [ ]:
# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code='alternatives',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.ClientInternal.*',
               supplier='Lusid',
               data_scope=scope,
               quote_type='Price',
               field='mid'),
            
                        models.MarketDataKeyRule(
               key='FX.CurrencyPair.*',
               supplier='Lusid',
               data_scope=scope,
               quote_type='Rate',
               field='mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='Lusid',
            credit='Lusid',
            equity='Lusid',
            fx='Lusid',
            rates='Lusid'),
        options=models.MarketOptions(
            default_supplier='Lusid',
            default_instrument_code_type='ClientInternal',
            default_scope=scope,
        attempt_to_infer_missing_fx=True)
        )
    )

upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request = models.UpsertRecipeRequest(
    configuration_recipe = configuration_recipe
    )
)